<a href="https://colab.research.google.com/github/4may/Transformer/blob/master/attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer

2018年にGoogle Brainから発表された新しいNNアーキテクチャ。attentionを使った構造が肝になる。

RNNと比較した時の、attentionのメリットは以下の通り。

* 並列計算可能であるため、GPUの恩恵を受けやすい
* データの時空間関係に依存せず、離れた位置にある情報同士の関係性も表現できる
* RNNのようなループ構造をもたず、構造がシンプル

逆に、デメリットは以下の通り。

* 時系列データの場合、ある時刻tの出力を計算をするのに全ての時刻の入力を必要とするため、非効率な場合がある

## Attention

Transformerを学ぶ前に、先にAttentionを学んでおこう。ここでは、attentionを使って日本語を英語に翻訳してみよう。

[Neural machine translation with attention](https://www.tensorflow.org/tutorials/text/nmt_with_attention)

### 必要なモジュールのインポート

In [0]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

### データの準備

データの前処理として、以下のことをしよう。

1. 各文にstart/end tokenを付け加える。
2. 各文から特殊文字を取り除く。
3. 単語と、単語のindexの変換辞書を作る
4. 各文をpaddingする

### 形態素解析

翻訳辞書内の日本語は分かち書きされていないため、形態素解析が必要。

google colab上に、辞書と必要なモジュールをインストールする。

In [2]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [3]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :   

動作確認してみよう。print("input Language: index to word mapping")
convert(input_lang, input_tensor_train[0])

In [4]:
import MeCab
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
sub = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
path = (sub.communicate()[0]).decode('utf-8')
m = MeCab.Tagger("-d {0}".format(path))
test = m.parse('今日は、良い気分になれる日だ！')
print(test)

今日	名詞,副詞可能,*,*,*,*,今日,キョウ,キョー
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
、	記号,読点,*,*,*,*,、,、,、
良い	形容詞,自立,*,*,形容詞・アウオ段,基本形,良い,ヨイ,ヨイ
気分	名詞,一般,*,*,*,*,気分,キブン,キブン
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
なれる	動詞,自立,*,*,一段,基本形,なれる,ナレル,ナレル
日	名詞,非自立,副詞可能,*,*,*,日,ヒ,ヒ
だ	助動詞,*,*,*,特殊・ダ,基本形,だ,ダ,ダ
！	記号,一般,*,*,*,*,！,！,！
EOS



In [5]:
lines = m.parse('今日は、良い気分になれる日だ！').split('\n')
#EOF, 空文字はスキップ
tokens = [line.split('\t')[0] for line in lines[:-2]]
tokens

['今日', 'は', '、', '良い', '気分', 'に', 'なれる', '日', 'だ', '！']

In [0]:
def mecab_tokenize(text):
  lines = m.parse(text).split('\n')
  tokens = [line.split('\t')[0] for line in lines[:-2]]
  return tokens

これで、分かち書きの準備ができた。

#### 翻訳辞書をダウンロード

[辞書のDL先](http://www.manythings.org/anki/)

In [14]:
#ローカル辞書をアップロードする
!unzip jpn-eng.zip

Archive:  jpn-eng.zip
  inflating: jpn.txt                 
  inflating: _about.txt              


In [0]:
path_to_file = 'jpn.txt'

In [0]:
def unicode_to_ascii(s):
  #日本語の場合、ユニコードカテゴリMnに該当する文字もnormalizeする必要がある。
  #そうでなければ、濁点や丸点まで消えてしまう。
  #return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
  return ''.join(c for c in unicodedata.normalize('NFD', s))

def preprocess_sentence(w):
  #形態素解析
  tokens = mecab_tokenize(w)
  w = ' '.join(tokens)

  #小文字に変え、かつ文の先頭と末尾の空白と改行を削除する。
  w = unicode_to_ascii(w.lower().strip())

  #単語と句読点との間にスペースを作る
  #彼 は、少年 です。 -> 彼 は 、 少年 です 。
  w = re.sub(r"([?.!,、。])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  w = w.strip()

  #<start> token、<end> tokenを付け加える
  #モデルの予測の開始場所と終了場所がわかるようにする。
  w = '<start> ' + w + ' <end>'
  return w

In [17]:
en_sentence = u"May I borrow this book?"
jp_sentence = u"この本を、借りてもいいですか?"
preprocess_sentence(en_sentence)

'<start> may i borrow this book ? <end>'

In [18]:
preprocess_sentence(jp_sentence)

'<start> この 本 を 、 借りて も いい です か ? <end>'

In [0]:
def create_dataset(path, num_examples):
  # 翻訳辞書の中身は以下のようになっている。
  # 最後の注釈はいらないので、落とす。
  # 英語\t日本語\t注釈\n
  # Go.	行け。	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #7421985 (Ninja)
  # Go.	行きなさい。	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #7421986 (Ninja)
  # Hi.	こんにちは。	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #373351 (tommy_san)
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:-1]] for l in lines[:num_examples]]
  
  return zip(*word_pairs)

In [20]:
en, jp = create_dataset(path_to_file, None)
en[-1]

"<start> i do many things at the same time , so not only am i reading things by akutagawa , i've also increased the amount of time i spend reading in english and i also read a little in german every day . <end>"

In [21]:
jp[-1]

'<start> 色々 並行 し て やっ てる から 芥川 ばかり 読ん でる の で も ない の だ よ 。 今 は 英語 読ん でる 時間 が 増え てる 。 ドイツ語 も 毎日 少し ずつ やっ てる 。 <end>'

In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, lang_tokenizer

In [0]:
def load_dataset(path, num_examples=None):
  tartget_lang, input_lang = create_dataset(path, num_examples)

  input_tensor, input_lang_tokenizer = tokenize(input_lang)
  target_tensor, tartget_lang_tokenizer = tokenize(tartget_lang)

  return input_tensor, target_tensor, input_lang_tokenizer, tartget_lang_tokenizer

## 実験の高速化のため、データセットのサイズに制限を設ける

100,000データだと多いので、30,000にしよう。

In [0]:
'''
データ数を絞る
'''
num_examples = 30000
input_tensor, target_tensor, input_lang, target_lang = load_dataset(path_to_file, num_examples)

In [25]:
'''
target tensorの最大長を取得
'''
max_length_target = target_tensor.shape[1]
max_length_input = input_tensor.shape[1]
max_length_target

16

trainingデータとvalidationデータを8:2の割合で分けてみよう

In [26]:
input_tensor_train, input_tensor_valid, target_tensor_train, target_tensor_valid = train_test_split(input_tensor, target_tensor, test_size=0.2)
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_valid), len(target_tensor_valid))

24000 24000 6000 6000


次に、単語と単語インデックスのマッピング関数を定義しよう。Attention layerは文字列ではなく数値を入力にとるため、文字列の数値変換が必要になる。

In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t != 0:
      print("%d ------> %s" % (t, lang.index_word[t]))

In [28]:
print("Input Language: index to word mapping")
convert(input_lang, input_tensor_train[0])
print()
print("Target Language: index to word mapping")
convert(target_lang, target_tensor_train[0])

Input Language: index to word mapping
1 ------> <start>
39 ------> あなた
4 ------> は
732 ------> お母さん
7 ------> に
845 ------> そっくり
46 ------> ね
3 ------> 。
2 ------> <end>

Target Language: index to word mapping
1 ------> <start>
7 ------> you
143 ------> look
85 ------> just
38 ------> like
34 ------> your
172 ------> mother
3 ------> .
2 ------> <end>


tensorflowのDatasetオブジェクトを作ろう。

In [0]:
BUFFER_SIZE = len(input_tensor_train) #24000
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE #375
embedding_dim = 256
units = 1024
#ボキャブラリ数 = 翻訳辞書の項目数であるため、inputもtargetも同じ値
vocab_input_size = len(input_lang.word_index) + 1 #10060
vocab_target_size = len(target_lang.word_index) + 1 #10060

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [30]:
ex_input_batch, ex_target_batch = next(iter(dataset))
ex_input_batch.shape, ex_target_batch.shape

(TensorShape([64, 27]), TensorShape([64, 16]))

これで、Attentionに渡すデータの準備ができた。

## Encoder, Decoder

machine translationの要である、encoderとdecoderを実装してみよう。

まずはencoderからだ。 encoderの構成は以下の通り。

* embedding
* GRU layer

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
    super(Encoder, self).__init__()
    self.batch_size = batch_size
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state
  
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_size, self.enc_units))

In [32]:
encoder = Encoder(vocab_input_size, embedding_dim, units, BATCH_SIZE)

sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(ex_input_batch, sample_hidden)
print('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 27, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


次にattentionを実装する。

attentionは、Luong attentionとBahdanau attentionの二種類が有名だ。両者の違いはattention scoreの計算方法である。前者はqueryとkeyの内積を計算する。後者はqueryとvalueの和を計算する。

* Luong attention : $score(q, k) = k^{T}Wq$
* Bahdanau attention : $score(q, k) = V^{T}tanh(W_1q + W_2v)$

In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
  
  def call(self, query, values):
    '''
    scoreを計算する
    '''
    #query hidden state shape = (batch_size, hidden_size) = (64, 1024)
    #quey_with_time_axis = (batch_size, 1, hidden_size) = (64, 1, 1024)
    #values shape = (batch_size, max_len, hidden_size) = (64, 27, 1024)
    #expand_dims()で、第二引数に指定したaxisの手前、つまりhidden_sizeの手前にサイズ１の次元を追加する。
    #scoreを計算するためには、valuesのshapeとqueryのshapeを合致させる必要がある。
    query_with_time_axis = tf.expand_dims(query, 1)

    densed_q = self.W1(query_with_time_axis) #(64, 1, 10)
    densed_v = self.W2(values) #(64, 27, 10)
    tanh_q_v = tf.nn.tanh(densed_q + densed_v) #(64, 27, 10)
    score = self.V(tanh_q_v) #(64, 27, 1)

    '''
    attention weightを計算する
    axisを指定しないと最後の次元に対してsoftmaxを計算してしまう。
    ここでは、max_length分だけの重みのsoftmaxを計算したいので、axis=1(番目の次元)を指定している
    '''
    attention_weights = tf.nn.softmax(score, axis=1) #(64, 27, 1)

    '''
    context_vectorを求める。
    context_vector shape = (batch_size, hidden_size)
    '''
    context_vector = attention_weights * values #(64, 27, 1024)
    #reduce_sumにより、axis=1次元目がなくなる
    context_vector = tf.reduce_sum(context_vector, axis=1) #(64, 1024)

    return context_vector, attention_weights

作成したattention layerの動作確認をしよう。

In [34]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch_size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch_size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 27, 1)


次は、decoderを実装しよう。decoderの構成は以下の通り。

* attention layer
* embedding layer
* GRU layer
* FC layer

通常のdecoderと異なる点は、decoderのembedding tensorにattention layerの出力tensorが加わる点だ。

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_size):
    super(Decoder, self).__init__()
    self.batch_size = batch_size
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.attention = BahdanauAttention(self.dec_units)
  
  def call(self, x, hidden, enc_output):
    '''
    attention layer
    '''
    #context vector : (batch_size, hidden_size)
    #attention weiths : (batch_size, sequence_length, 1)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    '''
    embedding layer
    '''
    #(batch_size, 1, embedding_dim)
    x = self.embedding(x) #(64, 1, 256)

    #-1番目(最後)の次元の要素数を増やす
    #(batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1) #(64, 1, 1280)

    '''
    GRU layer
    '''
    #output: (64, 1, 1024)
    #state: (64, 1024)
    output, state = self.gru(x)

    output = tf.reshape(output, (-1, output.shape[2])) #(64, 1024)

    #(batch_size, vocab)
    predictions = self.fc(output)

    return predictions, state, attention_weights

Decoderの動作確認をしよう。

In [36]:
decoder = Decoder(vocab_target_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)
print('Decoder output shape: (batch_size, vocab_size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab_size) (64, 6113)


### optimizerとloss functionを定義しよう

optimizerはAdamを使う。loss objectにはSparseCategoricalCrossentropyを使い、出力はlogitとする。なお、loss functionとしては、出力のlogitsにmaskを掛け合わせたものの、平均値とする。

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  #realが0でなければtrue, realが０であればfalse
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  #realが0ならlossの値も0になる
  loss_ *= mask

  return tf.reduce_mean(loss_)

## checkpointの定義

checkpointを作っておこう。

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

## Training

学習手順は以下の通り。

1. 入力データをencodeする(encoderに渡す)。戻り値として、output, hidden state(GRUの出力値)を取得する

2. encoder output, hidden state, decoder input(start token)をdecodeする。戻り値として、predictions, hidden stateを取得する。

3. decoderのhidden stateをモデルに戻す。そして、predictionsを使ってloss functionsを計算する

4. 「teacher」によって、target wordを次のdecoderのインプットとする。

5. gradientを計算して、optimizerに適用、逆伝搬させる。

In [0]:
@tf.function
def train_step(input, target, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    '''
    step1
    '''
    enc_output, enc_hidden = encoder(input, enc_hidden)

    '''
    step2-0
    '''
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang.word_index['<start>']] * BATCH_SIZE, 1)
    #全てのステップtごとにloss functionを計算する。
    for t in range(1, target.shape[1]):
      '''
      step2
      '''
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      '''
      step3
      '''
      loss += loss_function(target[:, t], predictions)

      '''
      step4
      '''
      #teacher forcing
      dec_input = tf.expand_dims(target[:, t], 1)
  
  #lossの平均値(batchごとのloss)
  batch_loss = (loss / int(target.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  '''
  step5
  '''
  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

それでは実際に学習を走らせてみよう。

In [1]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (input, target)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(input, target, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))

  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)
  print('Epoch {} Loss {:.4f}'.format(epoch+1, total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

NameError: ignored

Google colabのTPUインスタンスだと半日がかり・・・😱

## Translate

学習したモデルを使って、翻訳をしてみよう。

* evaluate関数は、teacher forcingをしないこと以外はtraining関数と同じ処理をする。
* <end> tokenを予測したらモデルは予測をストップする。
* 時刻tごとにattention weightsを保存する。
* encoder outputは一つの入力につき一度だけ計算されることに注意

In [0]:
def evaluate(sentence):  
  '''
  入力文字列(翻訳対象)に対する前処理
  '''
  sentence = preprocess_sentence(sentence)

  #['<start>', '今日', 'は', '良い', '天気', 'だ', '<end>']のようなトークン列が得られる
  tokens = sentence.split(' ')
  #tokenごとに、翻訳辞書内の何番目に存在するかを求める(stringをintegerにマッピングする)
  inputs = [input_lang.word_index[i] for i in tokens]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], 
                                                         maxlen=max_length_input,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  '''
  翻訳処理
  '''
  result = ''
  
  hidden = [tf.zeros((1, units))] #(1, 1024)
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([target_lang.word_index['<start>']], 0)

  attntion_plot = np.zeros((max_length_target, max_length_input)) #(16, 27)

  for t in range(max_length_target):
    predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_output)

    #一次元ベクトルに変形する
    attention_weights = tf.reshape(attention_weights, (-1,)) #(1728,)
    attention_plot[t] = attention_weights.numpy() 

    #翻訳後の単語インデックス
    predicted_id = tf.argmax(predictions[0]).numpy() #scalar(integer)
    #翻訳後の単語
    predicted_word = target_lang.index_word[prediction_id]

    result += predicted_word + ' '

    #最後まで翻訳したら終了
    if predicted_word == '<end>':
      return result, sentence, attention_plot
    
    dec_input = tf.expand_dims([predicted_id], 0) #(1, 1)
  
  return result, sentence, attention_plot

モデルの性能を評価するために、attention weightsをプロットしてみよう

In [0]:
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(1, 1, 1)
  #行列の中身をプロットする
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize':14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels(['' + predicted_sentence, fontdict=fontdict])

  ax.xaxis.set_major_locator(ticker.MutipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MutipleLocator(1))

  plt.show()

translate関数を定義しよう。

In [0]:
def translate(sentence):
  #翻訳処理
  result, sentence, attention_plot = evaluate(sentence)

  #翻訳文の表示
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  #attention weightの表示
  results = result.split(' ') #翻訳文の単語数
  sentences = mecab_tokenize(sentence) #入力文の単語数
  attention_plot = attentin_plot[:len(results), :len(sentences)]
  plot_attentoin(attention_plot, sentences, results)

## 翻訳

実際に翻訳してみよう!!

最新の学習結果から、学習モデルをロードしよう。

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

#### 例１

In [0]:
translate('今日はとても寒い。')

#### 例２

In [0]:
translate('これは私の人生だ。')

#### 例３

In [0]:
translate('あなたは、まだ家にいますか？')

#### 例４

In [0]:
translate('それを理解してみよう。')